In [1]:
import tensorflow as tf
import numpy as np
from tensorflow.keras import Model, layers, datasets, optimizers


In [15]:
mnist = datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()

x_train, x_test = x_train.astype(np.float32)/255, x_test.astype(np.float32)/255
x_train, x_test = np.expand_dims(x_train, axis=3), np.expand_dims(x_test, axis=3)


train = tf.data.Dataset.from_tensor_slices((x_train, y_train)).batch(256)
test = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(256)

In [16]:
class basic_model(Model):
    def __init__(self):
        super(basic_model, self).__init__()
        
        self.conv1 = layers.Conv2D(256, (3, 3), padding="same", input_shape=[28, 28, 1])
        self.pool1 = layers.MaxPool2D(pool_size=(2,2))
        self.conv2 = layers.Conv2D(128, (3, 3), padding="same")
        self.pool2 = layers.MaxPool2D(pool_size=(2,2))
        
        self.flatten = layers.Flatten()
        self.fc1 = layers.Dense(80, activation='relu')
        self.fc2 = layers.Dense(10)
        
    def call(self, inputs):
        x = self.conv1(inputs)
        x = self.pool1(x)
        x = self.conv2(x)
        x = self.pool2(x)
        x = self.flatten(x)
        x = self.fc1(x)
        x = self.fc2(x)
        return x 
    
class basic_model_2(Model):
    def __init__(self):
        super(basic_model, self).__init__()
        
        model = models.Sequential()
        model.add(layers.Dense(20))
        model.add(activation='relu')
        model.add(layers.Dense(10))

        
    def call(self, inputs):
        x = tf.convert_to_tensor(inputs)

        return self.model(x)

In [17]:
image_size = 28*28
num_epochs = 100
batch_size = 256
learning_rate = 1e-3

model = basic_model()
model.build(input_shape=(None, 28, 28, 1))
model.summary()
optimizer = optimizers.Adam(learning_rate=learning_rate)

Model: "basic_model_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_6 (Conv2D)            multiple                  2560      
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 multiple                  0         
_________________________________________________________________
conv2d_7 (Conv2D)            multiple                  295040    
_________________________________________________________________
max_pooling2d_7 (MaxPooling2 multiple                  0         
_________________________________________________________________
flatten_3 (Flatten)          multiple                  0         
_________________________________________________________________
dense_6 (Dense)              multiple                  501840    
_________________________________________________________________
dense_7 (Dense)              multiple                

In [ ]:
for epoch in range(num_epochs):
    for step, (x, y) in enumerate(train):
        with tf.GradientTape() as tape:
            logits = model(x)
            
            loss = tf.nn.sigmoid_cross_entropy_with_logits(labels=tf.one_hot(y, depth=10), logits=logits)
            
        gradients = tape.gradient(loss, model.trainable_variables)
        optimizer.apply_gradients(zip(gradients, model.trainable_variables))
        
        if step % 10 == 0:
            print(epoch, step, 'loss:', loss.numpy())
            
            
    model.save_weights('./my_model')

0 0 loss: [[3.13369201e-05 9.15374549e-05 1.82259567e-02 ... 3.64208757e-03
  3.06172878e-04 2.10835096e-05]
 [1.34916932e-04 1.51771800e-07 2.56671850e-03 ... 2.63250637e-04
  5.31944854e-04 4.69596671e-05]
 [1.32590387e-04 2.02970416e-03 3.28293099e-04 ... 4.16822237e-04
  1.47183817e-02 1.43906977e-02]
 ...
 [3.65889500e-05 4.17226431e-08 5.18199406e-04 ... 5.24613153e-09
  2.97228951e-04 3.55560026e-08]
 [9.48386453e-03 3.73202842e-03 3.15111451e-04 ... 7.03135538e-06
  4.06066477e-02 4.14541049e-04]
 [1.22168785e-05 4.44644457e-03 2.74007823e-02 ... 7.32306802e-08
  1.40369749e-02 6.60919625e-07]]
0 10 loss: [[2.34059644e-05 1.02985296e-05 3.52019447e-06 ... 7.80065893e-05
  1.13684116e-02 1.07203402e-01]
 [1.73302193e-04 9.92806008e-07 3.34716424e-05 ... 2.07229357e-04
  2.41120420e-02 1.06798514e-04]
 [6.09024719e-05 6.65502856e-04 1.52734356e-05 ... 1.63168870e-02
  6.33267174e-03 1.62345022e-01]
 ...
 [1.77303761e-01 8.27569480e-08 1.88711874e-05 ... 4.74969738e-06
  4.2621174

0 140 loss: [[7.0721311e-05 7.5138887e-05 1.6477117e-05 ... 1.5861944e-03
  4.4538105e-05 3.1752023e-03]
 [5.7365603e-05 1.4885576e-05 1.2258436e-03 ... 2.0063933e-07
  8.8779271e-02 6.7336122e-03]
 [5.0592067e-04 3.1780105e-02 3.3230212e-02 ... 1.4692035e-01
  1.1722461e-04 2.4103440e-02]
 ...
 [1.1645935e-08 7.5887993e-04 1.8205560e-06 ... 7.1092662e-03
  2.3506639e-04 1.9210802e-05]
 [6.8435846e-05 3.5212570e-04 1.2519928e-04 ... 1.3956826e-05
  5.2080047e-04 4.0285944e-05]
 [2.1161104e-03 5.3471206e-03 3.1375446e-04 ... 1.7060556e-02
  1.4494971e-04 2.5722773e-05]]
0 150 loss: [[3.13189230e-04 2.05578508e-05 5.73609541e-05 ... 2.21249437e-08
  7.36746779e-06 1.82126939e-10]
 [6.28886028e-06 9.67157732e-08 8.92576203e-03 ... 7.77410278e-08
  9.01685715e-01 1.59589337e-08]
 [1.68554434e-05 3.14339559e-05 1.32512268e-05 ... 6.26198074e-04
  1.02260022e-03 2.07054033e-03]
 ...
 [6.54260875e-05 4.71579995e-07 1.32101355e-02 ... 3.80675832e-04
  3.47794878e-04 1.23071266e-04]
 [9.9575532

1 40 loss: [[1.6034752e-03 1.2372755e-07 5.5663800e-04 ... 1.7277795e-04
  1.0827490e-04 1.0937901e-05]
 [1.3427312e-03 4.1792812e-04 3.9236684e-06 ... 3.7982341e-04
  7.1087733e-02 5.5493605e-03]
 [6.0357247e-04 7.1770863e-08 2.0894076e-05 ... 3.0086257e-03
  1.3129844e-04 1.2566565e-04]
 ...
 [8.6837371e-10 1.6138742e-06 4.1279759e-06 ... 1.3407161e-04
  7.3680158e-06 2.7983609e-07]
 [2.1431930e-07 1.0355385e-05 2.8603077e-07 ... 4.1557394e-04
  5.3756135e-03 8.5910661e-03]
 [1.9162127e-07 2.1303957e-07 1.4336799e-08 ... 2.7799683e-03
  9.7003125e-05 2.4118591e-03]]
1 50 loss: [[1.23980630e-03 6.84941964e-11 2.89698594e-08 ... 1.81934040e-04
  1.15618086e-03 2.25184504e-05]
 [8.08376444e-06 1.11073814e-05 3.37402416e-05 ... 4.95643260e-09
  2.08466896e-04 1.69366639e-11]
 [2.28774320e-06 1.26782396e-09 4.13240748e-04 ... 5.53395075e-04
  1.37002161e-03 1.56363328e-07]
 ...
 [2.61901319e-03 4.20627557e-03 5.73620375e-04 ... 4.09312472e-02
  7.32915578e-06 1.77478716e-01]
 [5.15115653e

1 180 loss: [[2.0473342e-07 3.3351846e-09 1.0435233e-06 ... 6.1319923e-05
  2.2158960e-05 9.4192357e-05]
 [1.8798141e-06 5.0419853e-06 5.3865890e-05 ... 2.2391545e-02
  2.2476927e-07 3.9983275e-05]
 [6.9152550e-10 2.0352306e-06 3.8688218e-03 ... 3.9575662e-02
  1.4845763e-07 7.3368568e-11]
 ...
 [2.9515080e-05 9.7636021e-06 3.8276994e-08 ... 1.3331039e-04
  7.1260321e-05 3.7422091e-05]
 [7.2011921e-05 8.0961880e-04 4.7760882e-06 ... 8.1744732e-04
  1.9102509e-04 2.3937611e-07]
 [2.5187837e-06 1.3156022e-03 3.5221427e-07 ... 7.9692289e-04
  8.4053242e-04 3.3757670e-09]]
1 190 loss: [[1.1709755e-06 2.8623480e-04 2.9849380e-05 ... 2.5154638e-06
  7.4576838e-07 2.2204684e-11]
 [1.9942638e-06 1.0886166e-06 1.4083955e-04 ... 1.7875101e-08
  1.0409715e-05 6.3388600e-11]
 [1.9941275e-07 1.4435174e-04 7.1677245e-04 ... 6.0511542e-07
  4.1831691e-02 1.9053248e-07]
 ...
 [8.1795542e-07 2.6963438e-05 9.4629551e-04 ... 2.3606415e-03
  4.0250136e-08 6.0066403e-05]
 [5.7134706e-01 1.0180738e-04 1.632

2 80 loss: [[4.52655058e-07 2.08008090e-11 1.91418536e-07 ... 1.50400536e-09
  1.74969989e-06 1.36452571e-10]
 [1.57720083e-03 5.36736388e-06 2.96030543e-04 ... 5.49011538e-08
  4.53311685e-07 1.95974707e-07]
 [1.33383751e-03 2.81955045e-06 6.43998384e-03 ... 1.07263654e-01
  8.52626181e-05 2.96523067e-05]
 ...
 [7.85456621e-07 1.32227224e-05 1.57748782e-05 ... 1.76983997e-10
  1.41922967e-04 6.59928926e-13]
 [1.23816903e-07 3.61809271e-09 3.80770402e-08 ... 1.50809018e-02
  9.04124172e-05 9.38950200e-03]
 [1.97432351e+00 1.96519323e-07 2.38593111e-06 ... 7.39880046e-03
  3.71261763e-07 5.01922941e+00]]
2 90 loss: [[3.2416928e-08 2.3012840e-08 8.7767658e-05 ... 2.2828253e-07
  1.4015246e-05 3.5397504e-13]
 [1.0214794e-03 2.0632641e-02 4.4926084e-05 ... 9.6476913e-07
  1.2307027e-05 2.0326877e-06]
 [3.5147230e-05 7.0226895e-07 4.9195037e-06 ... 2.3859362e-05
  9.2434289e-04 2.6873944e-04]
 ...
 [3.8301005e-05 3.9842897e-04 5.8705855e-06 ... 1.6040141e-06
  1.5243232e-08 2.7142855e-06]
 

...

7 170 loss: [[1.4379402e-13 6.0811908e-07 3.3474401e-09 ... 2.0718686e-07
  4.0148138e-09 3.0459130e-10]
 [1.0425931e-13 1.0313613e-12 2.0089110e-13 ... 9.9721065e-06
  3.8841194e-05 1.2366716e-06]
 [1.2569743e-09 1.8557203e-11 1.4841947e-03 ... 1.9893136e-08
  4.4455486e-05 1.5481922e-06]
 ...
 [1.8592251e-10 3.2762268e-09 4.1309531e-06 ... 6.2610379e-09
  3.4624941e-06 1.1763902e-08]
 [4.8285762e-13 9.6267729e-07 3.8401069e-09 ... 4.3005928e-09
  4.2705136e-12 5.3861529e-09]
 [7.9259212e-12 1.6388783e-07 5.8315566e-09 ... 1.0012904e-11
  7.1597304e-03 6.3723480e-09]]
7 180 loss: [[7.5644239e-12 5.1205734e-15 6.4448377e-11 ... 5.7104067e-08
  1.6699827e-08 6.4011141e-07]
 [1.0745367e-08 1.5342705e-07 5.9665435e-06 ... 3.6691099e-02
  6.9731412e-12 1.3570653e-06]
 [2.5440500e-16 3.1031916e-05 5.1824412e-05 ... 2.2906832e-05
  3.4878407e-15 4.5491509e-18]
 ...
 [1.1605649e-07 5.0768945e-09 7.0712490e-12 ... 1.7451161e-09
  2.9541420e-07 1.4042272e-06]
 [1.1473076e-06 3.2912585e-06 1.392

8 70 loss: [[1.6850807e-13 1.4058003e-15 7.3852182e-09 ... 1.6863244e-10
  4.7119215e-06 2.5823052e-08]
 [5.2479454e-08 9.2801106e-10 1.2409920e-09 ... 2.3272284e-07
  1.9810675e-07 3.3199899e-07]
 [5.5287666e-15 1.8253705e-09 1.4506964e-11 ... 4.5555469e-12
  1.9107522e-06 1.9972298e-11]
 ...
 [2.0648582e-11 4.0999802e-07 5.4318178e-08 ... 1.7171100e-06
  7.2816279e-06 2.3458077e-07]
 [6.2299588e-09 5.8050540e-07 8.9686904e-07 ... 3.3143713e-13
  3.9494498e-07 1.0073441e-10]
 [3.8173589e-09 5.5267006e-11 3.1069169e-07 ... 6.7236727e-10
  2.4988701e-06 3.2733269e-03]]
8 80 loss: [[2.09721780e-12 3.82140105e-17 4.04715476e-15 ... 1.12234504e-17
  4.38956024e-11 4.68647660e-16]
 [2.15388445e-05 3.14735868e-13 2.63768407e-09 ... 4.92039243e-10
  4.12699172e-12 5.87009019e-09]
 [2.18104833e-04 1.39179224e-09 8.69326755e-07 ... 2.01703926e-07
  2.85515259e-03 7.62690133e-10]
 ...
 [4.96471614e-11 4.28526034e-11 1.19079399e-10 ... 1.28635219e-18
  3.97657862e-09 7.68534670e-20]
 [5.06069037e

8 210 loss: [[9.0447639e-16 2.3337971e-06 7.5261503e-10 ... 2.8734832e-11
  6.2224144e-06 9.2984126e-05]
 [1.1759319e-07 7.5892831e-07 3.0619971e-09 ... 5.9055353e-08
  1.0741909e-06 2.1163978e-12]
 [1.1902160e-12 8.7744274e-07 4.6919271e-01 ... 7.4720400e-04
  3.7762206e-08 1.4226168e-05]
 ...
 [2.4363737e-11 1.5472001e-08 1.1977004e-10 ... 1.7993903e-08
  6.4767882e-08 1.2562643e-10]
 [8.1256557e-18 1.6784366e-05 1.0864975e-13 ... 5.3907808e-08
  6.2470363e-12 2.5640717e-16]
 [4.2224962e-10 1.7346789e-13 4.7538931e-11 ... 5.0013577e-10
  2.4899310e-10 7.2309461e-09]]
8 220 loss: [[4.4235432e-10 1.9472391e-05 3.7331285e-09 ... 5.3102792e-07
  6.2863631e-07 1.9654650e-05]
 [1.8566230e-11 2.8557943e-09 2.6957062e-07 ... 6.8738636e-08
  1.3080248e-08 2.7217062e-11]
 [3.7609871e-09 9.2061222e-13 7.6635573e-11 ... 2.9278588e-06
  3.5766620e-10 2.5666836e-13]
 ...
 [1.9586377e-03 3.4684317e-10 1.3922065e-09 ... 2.3879897e-17
  5.6887622e-08 7.7157330e-17]
 [4.9852837e-09 1.4732986e-07 3.629

9 110 loss: [[1.90715909e-07 2.91092844e-11 4.09992703e-04 ... 2.77377730e-08
  2.33869997e-07 1.54233294e-06]
 [1.09196936e-10 3.18534850e-15 1.34095784e-12 ... 4.37047967e-11
  1.43398911e-05 3.36345285e-09]
 [1.50086904e+00 3.63153405e-04 6.89810108e-18 ... 2.01301154e-09
  4.89360536e-05 3.18426313e-10]
 ...
 [7.52637277e-08 9.64950758e-11 4.95425367e-11 ... 5.27334805e-06
  2.96267821e-03 3.70443873e-02]
 [3.63502477e-06 7.34140870e-14 7.04249885e-08 ... 8.03921651e-11
  1.51994711e-04 1.19987544e-06]
 [2.26098314e-15 5.61545515e-12 1.98765843e-17 ... 1.05766112e-24
  9.87716703e-07 2.16077857e-17]]
9 120 loss: [[2.3118642e-12 7.2750490e-09 8.2992436e-04 ... 4.4557065e-04
  2.5052388e-04 1.8024181e-10]
 [2.6344745e-05 1.4966780e-07 1.7396923e-09 ... 3.5741812e-07
  1.7846918e-07 2.5175570e-10]
 [3.7276568e-09 1.0761586e-06 2.1472265e-06 ... 1.0731067e-06
  2.6240101e-09 1.5144702e-06]
 ...
 [8.9427549e-11 1.4679858e-09 5.5495619e-10 ... 7.2143350e-12
  2.1706361e-12 1.9481252e-09]

10 10 loss: [[4.9705747e-21 2.9741404e-10 2.3254243e-15 ... 2.9098929e-13
  1.3242254e-09 3.7079950e-08]
 [2.0851851e-07 6.4625422e-10 3.8679927e-11 ... 5.7032823e-10
  4.3882630e-11 1.4686109e-12]
 [3.1105576e-11 4.2754426e-13 1.2474802e-13 ... 9.1192101e-07
  4.8632348e-10 1.3873536e-08]
 ...
 [2.5786258e-14 6.8296837e-18 4.3745087e-18 ... 1.9231153e-14
  3.7007010e-13 1.2183500e-13]
 [9.3004404e-09 6.1537819e-10 5.0880230e-06 ... 3.4482888e-05
  2.0190609e-12 3.2927296e-08]
 [1.3161210e-15 7.0205517e-21 5.8162264e-13 ... 1.0588899e-05
  2.2377472e-07 8.8187662e-06]]
10 20 loss: [[1.4386544e-08 8.0022344e-20 3.1566544e-16 ... 6.8311352e-09
  3.3976326e-09 2.0355167e-13]
 [2.7651733e-11 8.2948547e-14 6.3981112e-15 ... 3.1938649e-23
  2.5369734e-10 1.1446554e-18]
 [3.4663282e-14 7.4101325e-10 2.6528251e-13 ... 3.2879109e-07
  3.1805740e-07 5.7272929e-01]
 ...
 [2.1395408e-08 4.7218652e-23 6.1279911e-21 ... 3.4153491e-10
  4.5635974e-07 1.2598780e-10]
 [2.3041174e-15 7.7107606e-17 3.434